In [2]:
"""
Get the packages we need to manipulate the dataset 
"""
import numpy as np 
import pandas as pd 
import os 
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture

In [3]:
"""
Specify the directories for data. These are universal for our repository-
this is the only thing you should not change.
"""

#rewrite this in the preferred way using os.join and os.path 

DATA_DIR = './' + 'raw_data'

DATA_NAME = '/CreditCard_data.csv'
"""
The sample Dataset summarizes the usage behavior of about
9000 active credit card holders during the last 6 months
"""

DATA_PATH = DATA_DIR + DATA_NAME

os.listdir(DATA_DIR)

if os.path.isfile(DATA_PATH): 
    print('DATA_PATH is good path')
else:
    raise ValueError('DATA_PATH is not valid path')


DATA_PATH is good path


In [5]:
"""
Define size of data sample to read-in. Use to_skip to define the logic. 

Import data as pandas dataframe for that format 
"""

SAMPLE_SIZE = 10000

def to_skip(index): 
    """
    Function to be used with pandas skiprows
    """
    keeprow = False
    
    if index <= SAMPLE_SIZE:
        keeprow = True
    else:
        keeprow = False
    
    return keeprow

raw_data = pd.read_csv(DATA_PATH,nrows = SAMPLE_SIZE)
                                              
print('Shape of the data is :', raw_data.shape, '\n')
print('The attributes of the data:\n', raw_data.columns)

Shape of the data is : (8950, 18) 

The attributes of the data:
 Index(['CUST_ID', 'BALANCE', 'BALANCE_FREQUENCY', 'PURCHASES',
       'ONEOFF_PURCHASES', 'INSTALLMENTS_PURCHASES', 'CASH_ADVANCE',
       'PURCHASES_FREQUENCY', 'ONEOFF_PURCHASES_FREQUENCY',
       'PURCHASES_INSTALLMENTS_FREQUENCY', 'CASH_ADVANCE_FREQUENCY',
       'CASH_ADVANCE_TRX', 'PURCHASES_TRX', 'CREDIT_LIMIT', 'PAYMENTS',
       'MINIMUM_PAYMENTS', 'PRC_FULL_PAYMENT', 'TENURE'],
      dtype='object')


In [6]:
"""
Examine a sample of the data
"""
raw_data.head(10)

,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,C10001,40.900749,0.818182,95.40,0.00,95.40,0.000000,0.166667,0.000000,0.083333,0.000000,0,2,1000.0,201.802084,139.509787,0.000000,12
1,C10002,3202.467416,0.909091,0.00,0.00,0.00,6442.945483,0.000000,0.000000,0.000000,0.250000,4,0,7000.0,4103.032597,1072.340217,0.222222,12
2,C10003,2495.148862,1.000000,773.17,773.17,0.00,0.000000,1.000000,1.000000,0.000000,0.000000,0,12,7500.0,622.066742,627.284787,0.000000,12
3,C10004,1666.670542,0.636364,1499.00,1499.00,0.00,205.788017,0.083333,0.083333,0.000000,0.083333,1,1,7500.0,0.000000,NaN,0.000000,12
4,C10005,817.714335,1.000000,16.00,16.00,0.00,0.000000,0.083333,0.083333,0.000000,0.000000,0,1,1200.0,678.334763,244.791237,0.000000,12
5,C10006,1809.828751,1.000000,1333.28,0.00,1333.28,0.000000,0.666667,0.000000,0.583333,0.000000,0,8,1800.0,1400.057770,2407.246035,0.000000,12
6,C10007,627.260806,1.000000,7091.01,6402.63,688.38,0.000000,1.000000,1.000000,1.000000,0.000000,0,64,13500.0,6354.314328,198.065894,1.000000,12
7,C10008,1823.652743,1.000000,436.20,0.00,436.20,0.000000,1.000000,0.000000,1.000000,0.000000,0,12,2300.0,679.065082,532.033990,0.000000,12
8,C10009,1014.926473,1.000000,861.49,661.49,200.00,0.000000,0.333333,0.083333,0.250000,0.000000,0,5,7000.0,688.278568,311.963409,0.000000,12
9,C10010,152.225975,0.545455,1281.60,1281.60,0.00,0.000000,0.166667,0.166667,0.000000,0.000000,0,3,11000.0,1164.770591,100.302262,0.000000,12


In [ ]:
"""
Process raw data according to the values which are missing. Define the features to keep based on initial report of
completeness and information
"""
# raw_data.columns
#
#
# to_keep = ['race','gender','age','weight','admission_type_id','discharge_disposition_id',\
#            'time_in_hospital','num_lab_procedures','num_procedures','num_medications',
#            'number_outpatient','number_emergency','number_inpatient','diag_1','diag_2']



In [7]:
"""
Define the features desired to keep, two group of features (in different scale) available:
1. Absolute amount in dollar
2. Rating of frequency \in R([0,1])

This cell is intended only for list manipulation - we actually slice the data in the next cell 
""" 

features = raw_data.columns

#Here are the features related to the absolute amount of money
#Some of them can be used as LABEL
ft_basic =['BALANCE', 'PURCHASES',
       'ONEOFF_PURCHASES', 'INSTALLMENTS_PURCHASES', 'CASH_ADVANCE',
       'CREDIT_LIMIT', 'PAYMENTS',
       'MINIMUM_PAYMENTS', 'PRC_FULL_PAYMENT']
# the last one is the percent of full payment being paid
# ft_basic_keep = ft_basic[:]

#Here are the features related to the frequency
ft_freq= ['BALANCE_FREQUENCY','PURCHASES_FREQUENCY', 'ONEOFF_PURCHASES_FREQUENCY',
       'PURCHASES_INSTALLMENTS_FREQUENCY', 'CASH_ADVANCE_FREQUENCY']

#Here are the features haven't been used
ft_unused = set(list(features))-set(ft_basic)-set(ft_freq)

# #Here are all the features we will keep or toss
# ft_keep = ft_basic_keep + ft_med_keep
# ft_toss = [x for x in features if (x not in ft_keep)]
#
print('Here are the features related to $$:\n', ft_basic,'\n')
print('Here are the features related to frequency: \n',ft_freq,'\n')
print('Here are the features not used: \n', ft_unused)

Here are the features related to $$:
 ['BALANCE', 'PURCHASES', 'ONEOFF_PURCHASES', 'INSTALLMENTS_PURCHASES', 'CASH_ADVANCE', 'CREDIT_LIMIT', 'PAYMENTS', 'MINIMUM_PAYMENTS', 'PRC_FULL_PAYMENT'] 

Here are the features related to frequency: 
 ['BALANCE_FREQUENCY', 'PURCHASES_FREQUENCY', 'ONEOFF_PURCHASES_FREQUENCY', 'PURCHASES_INSTALLMENTS_FREQUENCY', 'CASH_ADVANCE_FREQUENCY'] 

Here are the features not used: 
 {'CUST_ID', 'PURCHASES_TRX', 'TENURE', 'CASH_ADVANCE_TRX'}


In [8]:
"""
Subset the Raw Data: Here, we refine data according to the desired features to keep defined in the preivous cell. Then 
we type convert the columns in order to actually apply clustering to them. 
"""

data = raw_data[ft_basic]
ft_kept = data.columns
data.head(10)

,BALANCE,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT
0,40.900749,95.40,0.00,95.40,0.000000,1000.0,201.802084,139.509787,0.000000
1,3202.467416,0.00,0.00,0.00,6442.945483,7000.0,4103.032597,1072.340217,0.222222
2,2495.148862,773.17,773.17,0.00,0.000000,7500.0,622.066742,627.284787,0.000000
3,1666.670542,1499.00,1499.00,0.00,205.788017,7500.0,0.000000,NaN,0.000000
4,817.714335,16.00,16.00,0.00,0.000000,1200.0,678.334763,244.791237,0.000000
5,1809.828751,1333.28,0.00,1333.28,0.000000,1800.0,1400.057770,2407.246035,0.000000
6,627.260806,7091.01,6402.63,688.38,0.000000,13500.0,6354.314328,198.065894,1.000000
7,1823.652743,436.20,0.00,436.20,0.000000,2300.0,679.065082,532.033990,0.000000
8,1014.926473,861.49,661.49,200.00,0.000000,7000.0,688.278568,311.963409,0.000000
9,152.225975,1281.60,1281.60,0.00,0.000000,11000.0,1164.770591,100.302262,0.000000


In [15]:
"""
Clean the Data: Here we identify rows with missing fields in the desired categories. First, nan/None type is filtered and 
the row number is added to a list of rows which will be skipped when reading in the data for processing
"""

#Iterate over the list of desired features to keep; extract the series aech time, and build the list of rows associated
#to a missing data value available in that series. This is better than simply calling .dropna() one time 

rows_nan = list()
#This is an iterative approach by checking each column - this can likely be done across the whole array simultaneously 

for feature in ft_kept:
    feature_series = data[feature]
    missing_bool = feature_series.isnull()
    nan_indices = feature_series.index[missing_bool]
    if not nan_indices.empty:
        print("Issue Feature:\n", feature,'\n', nan_indices, '\n Num of null=', len(nan_indices), '\n\n')
        rows_nan += list(nan_indices)

# get rid of repetition and sort
rows_nan = list(set(rows_nan))
rows_nan.sort()

# Get rid of rows containing null or empty
data_clean = data.drop(rows_nan)

# value_dictionary = dict()
# for feature in ft_kept:
#     feature_series = data[feature]
#     unique_vals = list()
#     for x in feature_series:
#         if x in unique_vals:
#             pass
#         else:
#             unique_vals.append(x)
#     value_dictionary[feature] = unique_vals

#Here are the available values in each feature that are stored 
# print(value_dictionary)

Issue Feature:
 CREDIT_LIMIT 
 Int64Index([5203], dtype='int64') 
 Num of null= 1 


Issue Feature:
 MINIMUM_PAYMENTS 
 Int64Index([   3,   45,   47,   54,   55,   56,   63,   93,   94,   97,
            ...
            8753, 8755, 8803, 8838, 8854, 8919, 8929, 8935, 8944, 8946],
           dtype='int64', length=313) 
 Num of null= 313 




In [16]:
"""
Here we create a clusterable dataset by converting into Numpy array.
We also additionally extract one feature to be used as label
Return: X: (NxD) data ready to be clustered
        y: (Nx1) categorical label to examine clusters
"""
# specify one label from clean data feature
ft_label = 'CREDIT_LIMIT'
assert ft_label in ft_kept, "Label feature not in features being used"

# the remaining are the real data feature
ft_data = list(ft_kept).copy()
ft_data.remove(ft_label)
X = data_clean[ft_data].values.astype(np.float64)
# label
y = data_clean[ft_label].values
"""
Further grouping needs to be done to convert into categorical label
"""

assert not np.any(np.isnan(X)), "Data contains NaN"
assert np.all(np.isfinite(X)), "Data contains +/-Inf"

In [17]:
# hyperparameter
N = X.shape[0] #number of data
d = X.shape[1] #dimension
k = 10 # number of clusters

In [ ]:
"""
Pipeline for Kmeans clustering given input data X
"""
kmeans = KMeans(n_clusters=k, random_state=0).fit(X)
center = kmeans.cluster_centers_

In [ ]:
%matplotlib inline
plt.figure(0)
plt.plot(np.tile(np.arange(d),(k,1)).T,center.T, linewidth = 2)
plt.xlabel('Feature Dimension', fontsize=14, fontweight='bold')
plt.ylabel('Amount of money',fontsize=14, fontweight='bold')
plt.title('KMeans-Clustering using {} centers'.format(k),fontsize=16, fontweight='bold')

In [ ]:
"""
Pipeline for GMM clustering given input data X
"""
gm = GaussianMixture(n_components=k, random_state=0).fit(X)
gm_center = gm.means_

In [ ]:
plt.figure(0)
plt.plot(np.tile(np.arange(d),(k,1)).T,gm_center.T, linewidth = 2)
plt.xlabel('Feature Dimension', fontsize=14, fontweight='bold')
plt.ylabel('Amount of money',fontsize=14, fontweight='bold')
plt.title('GMM-Clustering using {} centers'.format(k),fontsize=16, fontweight='bold')

